In [1]:
# #Numpy - for scientific computing
import numpy as np
# import scipy

# #For image reading and analysis
import glob
# import skimage.io
# import skimage.filters as sf
# from skimage import io, measure, filters, transform, morphology

# #Matplotlib plotting packages
import matplotlib as mpl
import matplotlib.pyplot as plt


# For loading bars
from tqdm.notebook import tqdm as tqdm

# # For creating DataFrames
import pandas as pd

In [2]:
import gliding_pkg.gliding_anal as gpkg

In [3]:
def expparam_adder(df_good, param_df):
    zeros = np.zeros(len(df_good))
    df_good['Motor Type'] = [param_df.loc['Motor_Type']['Value']]*len(df_good)
    df_good['Motor_Conc_uM'] = zeros + float(param_df.loc['Motor_Conc_uM']['Value'])
    df_good['MT_Conc_uM'] = zeros + float(param_df.loc['MT_Conc_uM']['Value'])
    df_good['ATP_Conc_uM'] = zeros + float(param_df.loc['ATP_Conc_uM']['Value'])
    df_good['ADP_Conc_uM'] = zeros + float(param_df.loc['ADP_Conc_uM']['Value'])
    df_good['P_Conc_uM'] = zeros + float(param_df.loc['P_Conc_uM']['Value'])
    df_good['Flag_Conc_mg-ml'] = zeros + float(param_df.loc['Flag_Conc_mg-ml']['Value'])
    df_good['BSA_Caesin_Both'] = [param_df.loc['BSA_Caesin_Both']['Value']]*len(df_good)
    df_good['Acq_Time_s'] = zeros + float(param_df.loc['Acq_Time_s']['Value'])
    df_good['Time_Interval_s'] = zeros + float(param_df.loc['Time_Interval_s']['Value'])
    df_good['Cy5_exptime_ms'] = zeros + float(param_df.loc['Cy5_exptime_ms']['Value'])
    df_good['Cy5_gain'] = zeros + float(param_df.loc['Cy5_gain']['Value'])
    df_good['YFP_exptime_ms'] = zeros + float(param_df.loc['YFP_exptime_ms']['Value'])
    df_good['YFP_gain'] = zeros + float(param_df.loc['YFP_gain']['Value'])
    df_good['Temperature'] = zeros + float(param_df.loc['Temperature']['Value'])
    
    return df_good

In [4]:
file_path = '../../data/gliding/'
file_folder = '2023-11-01_Gliding/1_phosphate_titration/2_Vary_phosphate_at_40uMATP/0mM_Pi'

In [5]:
param_df = pd.read_excel(file_path + file_folder + '/Exp_Params_Template.xlsx', names=['Param', 'Value']).set_index('Param')

In [6]:
files = np.sort(glob.glob(file_path + file_folder + '/TIRF*'))

In [7]:
# Params
acqu_interval = 1 #s acquisition intervals
umperpixel=0.161

In [15]:
file

'../../data/gliding/2023-11-01_Gliding/1_phosphate_titration/2_Vary_phosphate_at_40uMATP/0mM_Pi/TIRF_100x_1sInterval_1nMNCD_40uMATP_0mMADP_0mMPi_monoK_0mMKCl_1to50Antiflag_ilid_3'

In [17]:
file.split('/')[4] + '/' + file_folder.split('/')[1]+ file.split('/')[6]+ '_'+file.split('/')[7] #+ file_folder.split('/')[1] + file.split('/')[6]

'2023-11-01_Gliding/1_phosphate_titration2_Vary_phosphate_at_40uMATP_0mM_Pi'

In [22]:
#Lets pick 50 frames
start=0
stop=51

for file in tqdm(files):
    im_stack = gpkg.stackimport(file+'/Pos0/img*')
    im_max = np.amax(im_stack[start:stop], axis=0)
    im_thresh = gpkg.thresh_processing(im_max)
    im_label, unique_regions, region_counts = gpkg.MTregions(im_thresh) 
    im_internal = gpkg.edgeremove(im_label)
    im_sized = gpkg.smallremove(im_internal, unique_regions, region_counts)
    im_relabel = gpkg.makeskele(im_sized)
    im_cleaned = gpkg.onlylines(im_relabel)
    df = gpkg.makedf(im_cleaned, im_stack, start, stop)
    df = gpkg.speedcalculator(df, im_stack, start, stop)
    df_good = gpkg.goodtracks(df)
    df_good_MTlen = gpkg.MTlenths(df_good, im_stack, start, stop, umperpixel)
    df_final = expparam_adder(df_good_MTlen, param_df)
    print(df_final['speed (nm/s)'][0:10])
    df_final.to_csv('../../analyzed_data/gliding/' + file.split('/')[4] + '/' + file_folder.split('/')[1]+ file.split('/')[6]+ '_'+file.split('/')[7]+ '_start' + str(start) + '_stop' + str(stop) + '.csv', sep=',')

  0%|          | 0/3 [00:00<?, ?it/s]

/Users/anaduarte/RPgroup/am_atp/analysis/gliding/gliding_pkg/gliding_anal.py:245: RuntimeWarning: invalid value encountered in long_scalars
  com = np.sum(row_vals*xindex)/np.sum(row_vals)
/Users/anaduarte/RPgroup/am_atp/analysis/gliding/gliding_pkg/gliding_anal.py:353: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_good = df[df['r^2']>r2thresh][df['skele r^2']>skelethresh]
/Users/anaduarte/RPgroup/am_atp/analysis/gliding/gliding_pkg/gliding_anal.py:384: RuntimeWarning: divide by zero encountered in true_divide
  checkvals = (((x-x0)*np.cos(orientation) + (y-y0)*np.sin(orientation))**2 / major_len**2) + (((x-x0)*np.sin(orientation) - (y-y0)*np.cos(orientation))**2 / minor_len**2)


0     56.061026
1     14.184027
2     61.705862
3     28.280633
4     32.105294
5     47.892036
6     34.707553
9     60.225656
10    56.248009
11    62.076063
Name: speed (nm/s), dtype: float64


/Users/anaduarte/RPgroup/am_atp/analysis/gliding/gliding_pkg/gliding_anal.py:245: RuntimeWarning: invalid value encountered in long_scalars
  com = np.sum(row_vals*xindex)/np.sum(row_vals)
/Users/anaduarte/RPgroup/am_atp/analysis/gliding/gliding_pkg/gliding_anal.py:289: RankWarning: Polyfit may be poorly conditioned
  skele_xvals, skele_yvals, skele_params, skele_speed, skele_rsqrd = skeletonfit(kymo_thresh, pixel_size)
/Users/anaduarte/RPgroup/am_atp/analysis/gliding/gliding_pkg/gliding_anal.py:233: RuntimeWarning: divide by zero encountered in double_scalars
  rsqrd = 1-(ss_res/ss_tot)
/Users/anaduarte/RPgroup/am_atp/analysis/gliding/gliding_pkg/gliding_anal.py:255: RuntimeWarning: divide by zero encountered in double_scalars
  mfit = 1/mfit_inv
/Users/anaduarte/RPgroup/am_atp/analysis/gliding/gliding_pkg/gliding_anal.py:256: RuntimeWarning: divide by zero encountered in double_scalars
  bfit = - bfit_inv/mfit_inv
/Users/anaduarte/RPgroup/am_atp/analysis/gliding/gliding_pkg/gliding_a

1     44.730317
4     33.108871
5     52.161086
6     53.992732
10    58.513665
13    75.355000
14    96.283680
16    32.315000
17    23.112745
18    54.193620
Name: speed (nm/s), dtype: float64


/Users/anaduarte/RPgroup/am_atp/analysis/gliding/gliding_pkg/gliding_anal.py:245: RuntimeWarning: invalid value encountered in long_scalars
  com = np.sum(row_vals*xindex)/np.sum(row_vals)
/Users/anaduarte/RPgroup/am_atp/analysis/gliding/gliding_pkg/gliding_anal.py:224: RuntimeWarning: divide by zero encountered in double_scalars
  mfit = 1/mfit_inv
/Users/anaduarte/RPgroup/am_atp/analysis/gliding/gliding_pkg/gliding_anal.py:225: RuntimeWarning: divide by zero encountered in double_scalars
  bfit = - bfit_inv/mfit_inv
/Users/anaduarte/RPgroup/am_atp/analysis/gliding/gliding_pkg/gliding_anal.py:231: RuntimeWarning: invalid value encountered in add
  ss_res =  np.sum((yvals - (mfit*xvals+bfit))**2)
/Users/anaduarte/RPgroup/am_atp/analysis/gliding/gliding_pkg/gliding_anal.py:289: RankWarning: Polyfit may be poorly conditioned
  skele_xvals, skele_yvals, skele_params, skele_speed, skele_rsqrd = skeletonfit(kymo_thresh, pixel_size)
/Users/anaduarte/RPgroup/am_atp/analysis/gliding/gliding_pk

1     17.867577
3     43.513710
4     55.330090
6     49.327195
10    35.099457
11    67.700136
12    33.883426
14    28.338914
17    52.822084
18    27.712398
Name: speed (nm/s), dtype: float64


In [169]:
start=50
stop=100

for file in tqdm(files):
    im_stack = gpkg.stackimport(file+'/Pos0/img*')
    im_max = np.amax(im_stack[start:stop], axis=0)
    im_thresh = gpkg.thresh_processing(im_max)
    im_label, unique_regions, region_counts = gpkg.MTregions(im_thresh) 
    im_internal = gpkg.edgeremove(im_label)
    im_sized = gpkg.smallremove(im_internal, unique_regions, region_counts)
    im_relabel = gpkg.makeskele(im_sized)
    im_cleaned = gpkg.onlylines(im_relabel)
    df = gpkg.makedf(im_cleaned, im_stack, start, stop)
    df = gpkg.speedcalculator(df, im_stack, start, stop)
    df_good = gpkg.goodtracks(df)
    df_good_MTlen = gpkg.MTlenths(df_good, im_stack, start, stop, umperpixel)
    df_final = expparam_adder(df_good_MTlen, param_df)
    df_final.to_csv('../../analyzed_data/gliding/' + file.split('/')[4] + '/' + file_folder.split('/')[1] + file.split('/')[6] + '_start' + str(start) + '_stop' + str(stop) + '.csv', sep=',')

  0%|          | 0/3 [00:00<?, ?it/s]

/Users/anaduarte/RPgroup/am_atp/analysis/gliding/gliding_pkg/gliding_anal.py:245: RuntimeWarning: invalid value encountered in long_scalars
  com = np.sum(row_vals*xindex)/np.sum(row_vals)
/Users/anaduarte/RPgroup/am_atp/analysis/gliding/gliding_pkg/gliding_anal.py:353: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_good = df[df['r^2']>r2thresh][df['skele r^2']>skelethresh]
/Users/anaduarte/RPgroup/am_atp/analysis/gliding/gliding_pkg/gliding_anal.py:384: RuntimeWarning: divide by zero encountered in true_divide
  checkvals = (((x-x0)*np.cos(orientation) + (y-y0)*np.sin(orientation))**2 / major_len**2) + (((x-x0)*np.sin(orientation) - (y-y0)*np.cos(orientation))**2 / minor_len**2)
/Users/anaduarte/RPgroup/am_atp/analysis/gliding/gliding_pkg/gliding_anal.py:245: RuntimeWarning: invalid value encountered in long_scalars
  com = np.sum(row_vals*xindex)/np.sum(row_vals)
/Users/anaduarte/RPgroup/am_atp/analysis/gliding/gliding_pkg/gliding_anal.py:289: RankWa